In [10]:
with open('Wnt_proteinA.txt', 'r') as f:
    all_lines_A = f.readlines()
with open('Wnt_proteinB.txt', 'r') as f:
    all_lines_B = f.readlines()

In [11]:
ppi_pairs = []
for i in range(96):
    p1 = all_lines_A[i*2].rstrip()[1:]
    p2 = all_lines_B[i*2].rstrip()[2:]
    ppi_pair = p1 + '\t' + p2 + '\t1\n'
    ppi_pairs.append(ppi_pair)

In [12]:
ppi_pairs

['VIM\tFAT5\t1\n',
 'CGI-125\tFAT5\t1\n',
 'NFAT5\tPP3CA\t1\n',
 'PPP3CA\tLCB1\t1\n',
 'PLCB1\tAMK2A\t1\n',
 'PLCB1\tRKCA\t1\n',
 'PLCB1\tZD10\t1\n',
 'FZD10\tVL1\t1\n',
 'FZD10\tNT4\t1\n',
 'FZD10\tFRP1\t1\n',
 'FZD10\tNT9A\t1\n',
 'FZD10\tRP6\t1\n',
 'WNT4\tFRP1\t1\n',
 'WNT4\tER1\t1\n',
 'WNT4\tIF1\t1\n',
 'WNT4\tER1\t1\n',
 'SFRP1\tNT9A\t1\n',
 'SFRP1\tNT16\t1\n',
 'CER1\tNT9A\t1\n',
 'WNT9A\tIF1\t1\n',
 'WNT4\tROCN\t1\n',
 'WNT9A\tROCN\t1\n',
 'WIF1\tNT16\t1\n',
 'PROCN\tNT16\t1\n',
 'WNT16\tTRN4\t1\n',
 'WNT16\tZD10\t1\n',
 'LRP6\tKK1\t1\n',
 'DVL1\tY6H\t1\n',
 'DVL1\tOC307255\t1\n',
 'DVL1\tKD1\t1\n',
 'DVL1\tENP2\t1\n',
 'SENP2\tDK5RAP2\t1\n',
 'DVL1\tXXC4\t1\n',
 'DVL1\tSNK2A1\t1\n',
 'CSNK2A1\tHX1\t1\n',
 'DVL1\tPC2\t1\n',
 'DVL1\tRAT1\t1\n',
 'DVL1\tSK3B\t1\n',
 'DVL1\tXIN1\t1\n',
 'DVL1\tAAM1\t1\n',
 'DVL1\tAC4\t1\n',
 'DVL1\tANGL2\t1\n',
 'DVL1\tRICKLE1\t1\n',
 'DVL1\tSP13\t1\n',
 'PRICKLE1\tANGL2\t1\n',
 'DDX19\tKP1A\t1\n',
 'SKP1A\tBXO34\t1\n',
 'RAC4\tARK2\t1\n',
 'RAC4

In [14]:
prot2seq = {}
for i in range(96):
    id1 = all_lines_A[i*2].rstrip()[1:]
    seq1 = all_lines_A[i*2+1].rstrip()
    
    id2 = all_lines_B[i*2].rstrip()[1:]
    seq2 = all_lines_B[i*2+1].rstrip()
    
    if id1 not in prot2seq.keys():
        prot2seq[id1] = seq1
    if id2 not in prot2seq.keys():
        prot2seq[id2] = seq2
        

In [24]:
with open('seqs.fasta', 'r') as f:
    all_seqs = f.readlines()

In [28]:
protID2seqs = {}
for line in all_seqs:
    if line.startswith('>'):
        protID = line.rstrip().split('|')[1]
        protID2seqs[protID] = ''
    else:
        protID2seqs[protID] += line.rstrip()
        
    

In [30]:
prot_name2id = {}
find_id_list = []
for key1, value1 in prot2seq.items():
    for key2, value2 in protID2seqs.items():
        if value1 == value2:
            prot_name2id[key1] = key2
            find_id_list.append(key1)
            break

In [16]:
unique_keys = list(prot2seq.keys())

In [34]:
set(unique_keys)- set(find_id_list)

{'ACP5',
 'CDK5RAP2',
 'CSNK1A1',
 'CSNK2A1',
 'CTNNBIP1',
 'DDX19',
 'LOC307255',
 'PROCN',
 'RAC4',
 'RSTS'}

In [50]:
prot_name2id['ACP5'] = 'P13686'
prot_name2id['CDK5RAP2'] = 'Q05CZ3'
prot_name2id['CSNK1A1'] = 'Q6PJ06'
prot_name2id['CSNK2A1'] = 'Q5U5J2'
prot_name2id['CTNNBIP1'] = 'Q8WZA0'
prot_name2id['DDX19'] = 'Q9NUU7'
prot_name2id['LOC307255'] = ''
prot_name2id['PROCN'] = 'Q6P2Q9'
prot_name2id['RAC4'] = 'Q67VP4'
prot_name2id['RSTS'] = 'Q92793'

In [51]:
import requests, sys
import json
from tqdm import tqdm
ppi_go_terms = {}
remain_proteins = []
for t in tqdm(unique_keys):
    protID = prot_name2id[t]
    if protID !='':
        requestURL = "https://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId="+protID
        r = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if r.ok:
            responseBody = r.text
            data=json.loads(responseBody)
            for k in data['results']: 
                go_id = k['goId'] 
                if t not in ppi_go_terms.keys():
                    ppi_go_terms[t] = go_id
                else:
                    ppi_go_terms[t] = ppi_go_terms[t] + ';' + go_id
            if len(data['results']) == 0:
                remain_proteins.append(t)
        else:
            remain_proteins.append(t)
    else:
        ppi_go_terms[t] = ''

100%|██████████| 77/77 [02:19<00:00,  1.81s/it]


In [52]:
remain_proteins

[]

In [53]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [54]:
save_dict(ppi_go_terms,'prot2go.pkl')

In [55]:
save_dict(prot2seq, 'prot2seq.pkl')

In [56]:
ppi_go_terms['FAT5']

KeyError: 'FAT5'